# Seizure Prediction with EEG Data

Authors: Erin Shappell, Ben Shealy

In this notebook we demonstrate how to use electroencephalogram (EEG) data to predict seizures in epileptic and neurotypical patients. This work contains the code used for Erin Shappell's Honors Thesis submitted in Spring 2020.

In particular we draw from two EEG datasets:

- [TUH EEG Corpus](https://www.isip.piconepress.com/projects/tuh_eeg/html/downloads.shtml)
- [CHB-MIT Scalp EEG Database](https://archive.physionet.org/pn6/chbmit/)

The raw data for this notebook can be obtained from these two websites.

## Getting Started

In [ ]:
import glob
import json
import os
import random
import time

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import mne
import numpy as np
import pandas as pd
import sklearn
import sklearn.metrics
import sklearn.preprocessing

from tensorflow import keras

## Download Raw Data

In this notebook we will be working primarily with the CHB-MIT dataset. The raw data can be downloaded directly from their website. We will use only the first five patients.

In [ ]:
%%bash

DIRS="chb01 chb02 chb03 chb04 chb05"

for DIR in ${DIRS}; do
    wget -r -A edf -np "https://archive.physionet.org/pn6/chbmit/${DIR}"
done

mv archive.physionet.org/pn6/chbmit .
rm -rf archive.physionet.org

## Prepare Data

The CHB-MIT dataset contains data from many inviduals, but we will use only the first five. Each individual has approximately 40 recordings saved as EDF files, and each file has 23 channels associated with different areas of the brain. Additionally, each EDF file either contains or doesn't contain a single seizure event.

We need to convert this data from EDF format to CSV format so that we can load it with pandas. Additionally, even with only the first five individuals there is still significantly more data than what we need for our prediction task, so we will extract a 50 second window from each EDF file. In particular we will extract the seizure event if it is present or the first 50 seconds if there is no seizure event. This way we will have plenty of positive and negative samples.

__NOTE__: __You only need to run this code block once.__ The code will save a CSV file called `seizure.csv`, as well as individual CSV files for each channel. It is those CSV files that we will use for seizure prediction.

In [ ]:
# data summary: 
# chb01: 46 files, seizures at 3, 4, 15, 16, 18, 21, 26
# chb02: 35 files, seizures at 16, 19
# chb03: 38 files, seizures at 1, 2, 3, 4, 34, 35, 36
# chb04: 43 files, seizures at 5, 8, 28
# chb05: 39 files, seizures at 6, 13, 16, 17, 22

# sampling rate (in Hz)
s_rate = 256 

# Length (in seconds) of extraction window
w_size = 50

# indices of files that contain seizures
s_loc = [
    [3, 4, 15, 16, 18, 21, 26],
    [16, 19],
    [1, 2, 3, 4, 34, 35, 36],
    [5, 8, 28],
    [6, 13, 16, 17, 22]
]

# lower bounds of seizures
s_begin = [
    [2996, 1467, 1732, 1015, 1720, 327, 1862],
    [130, 3369],
    [362, 731, 432, 2162, 1982, 2592, 1725],
    [7804, 6446, 1679],
    [417, 1086, 2317, 2451, 2348]
]

# upper bounds of seizures (original, not used)
# s_end = [
#     [3036, 1494, 1772, 1066, 1810, 420, 1963],
#     [212, 3378],
#     [414, 796, 501, 2214, 2029, 2656, 1778],
#     [7853, 6557, 1781],
#     [532, 1196, 2413, 2571, 2465]
# ]

# initialize output dataframe
df_extracted = pd.DataFrame()

# process edf data from each patient directory
s_i = 0

for i in range(len(s_loc)):
    # get directory path
    file_path = 'chbmit/chb%02d' % (i + 1)
    
    # read all edf files in directory
    edf_files = glob.glob(os.path.join(file_path, '*.edf'))
    
    for filename in edf_files:
        print('Reading %s, s_i=%d' % (filename, s_i))

        # read edf file
        edf = mne.io.read_raw_edf(filename, verbose=False)

        # create dataframe from edf data
        df_raw = pd.DataFrame(data=edf.get_data().T, columns=edf.ch_names)

        # extract index from filename
        j = int(filename.replace('.', '_').split('_')[-2])

        # extract seizure data if it is present
        if j in s_loc[i]:
            s_idx = s_loc[i].index(j)
            begin = s_begin[i][s_idx]

        # otherwise extract from the beginning
        else:
            begin = 0

        # read each channel into dataframe
        for channel in df_raw.columns:
            # insert_data.reset_index(drop=True, inplace=True)
            df_extracted['%s_%d' % (channel, s_i)] = df_raw.loc[begin * s_rate : (begin + w_size) * s_rate - 1, channel]

        # rename weird column name
        df_extracted.rename(columns={ '# FP1-F7_%d' % (s_i) : 'FP1-F7_%d' % (s_i) }, inplace=True)

        # increment global index
        s_i += 1

# save extracted data to csv file
df_extracted.to_csv('seizure.csv', index=False, float_format='%.8g')

Even with only five individuals, our extracted dataset will be fairly large. Feel free to use more or fewer individuals, you just need to add the approriate information to the variables `s_loc` and `s_begin`. You can also experiment with the data extraction code: changing the window size, taking more time slices from each file at different starting points, etc.

The `seizure.csv` file contains all of the data that we'll use for the rest of this notebook, but it will also be helpful to have separate CSV files for each channel.

In [ ]:
# extract channel names from extracted dataframe
channels = [c.split('_')[0] for c in df_extracted.columns]
channels = list(set(channels))

# save separate csv files for each channel
for channel in channels:
    print(channel)

    # extract columns that belong to channel
    columns = [name for name in df_extracted.columns if name.startswith(channel)]

    # create dataframe for channel
    df_channel = df_extracted[columns]
    df_channel.columns = [c.split('_')[1] for c in df_channel.columns]

    # save dataframe to csv file
    df_channel.to_csv('seizure.%s.csv' % (channel), index=False, float_format='%.8g')

## CNN Seizure Classifier

The first task that we will explore is using a convolutional neural network (CNN) to predict whether a time series sample represents a seizure event or not. CNNs are typically used for image classification, and they use 2D convolution because images have 2-dimensional structure. However, our time series is 1-dimensional, so we will use 1D convolution.

Now you might think, isn't our time series data 2-dimensional since there are also channels? Yes, our data is 2-dimensional in the sense that each data sample is a 12,800 x 23 time series. However, the 23 channels in our data are independent of each other, so it would not make sense to convolve _across_ channels. Think of it in another way -- does it matter how the channels are ordered? By constrast, does it matter how the rows or columns in an image are ordered? That is the key difference here.

First we define a few functions we will use to automate the train/test cycle for our CNN model:

In [ ]:
def train_test_split(X, y, train_size=0.8, n_timesteps=12800):
    # compute split index
    split_index = int(X.shape[0] * train_size)

    # extract train set
    X_train = X[:split_index, :n_timesteps]
    y_train = y[:split_index]

    # extract test set
    X_test = X[split_index:, :n_timesteps]
    y_test = y[split_index:]

    return X_train, y_train, X_test, y_test

def create_cnn(X_train, y_train, X_test, y_test):
    # get data dimensions
    n_timesteps = X_train.shape[1]
    n_channels = X_train.shape[2]

    # create model
    model = keras.models.Sequential()
    model.add(keras.layers.Conv1D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(n_timesteps, n_channels)))
    model.add(keras.layers.Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.MaxPooling1D(pool_size=2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(100, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

def run_experiment():
    # create train/test data
    X_train, y_train, X_test, y_test = train_test_split(X, y)

    # create model
    model = create_cnn(X_train, y_train, X_test, y_test)

    # print model summary
    model.summary()

    # train model
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=True)

    # evaluate model
    score = model.evaluate(X_test, y_test)

    # print test accuracy for test set
    print('test accuracy: %f' % (score[1]))
    
    # plot the training accuracy
    plt.figure(figsize=(20, 6))
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Training Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'], loc='upper left')
    plt.show()

    # plot the training loss
    plt.figure(figsize=(20, 6))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Training Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'], loc='upper left')
    plt.show()

Now we must load the data and labels into a format that can be consumed by our CNN. For the data, we need to load our channel dataframes into a single 3D Numpy array with dimensions (n_samples x n_timesteps x n_channels).

In [ ]:
# note: remove file 'seizure.ECG.csv' as it is not used and will cause an error
# !rm -f seizure.ECG.csv

# load csv files for each channel
csv_files = glob.glob('seizure.*.csv')
channels = []

for filename in csv_files:
    print(filename)

    channel = pd.read_csv(filename)
    channel = channel.values.T
    channels.append(channel)

# stack channels into 3-d numpy array
X = np.dstack(channels)

# print data dimensions (n_samples x n_timesteps x n_channels)
print(X.shape)

For the labels, we have the list of indices of positive examples (seizures), we just need to transform this list into a single vector of labels for each sample.

In [ ]:
# indices of files that contain seizures
s_loc = [
    [3, 4, 15, 16, 18, 21, 26],
    [16, 19],
    [1, 2, 3, 4, 34, 35, 36],
    [5, 8, 28],
    [6, 13, 16, 17, 22]
]

# compute labels from indices
y = []

for i in range(len(s_loc)):
    # get directory path
    file_path = 'chbmit/chb%02d' % (i + 1)

    # read all edf files in directory
    edf_files = glob.glob(os.path.join(file_path, '*.edf'))
    
    for filename in edf_files:
        # extract index from filename
        j = int(filename.replace('.', '_').split('_')[-2])

        # label positive example (seizure)
        if j in s_loc[i]:
            y.append(1)

        # label negative example (no seizure)
        else:
            y.append(0)

# print label stats                
print("Number of positive examples: %d" % (y.count(1)))
print("Number of negative examples: %d" % (y.count(0)))

# convert labels to numpy array
y = np.array(y)

Now we are ready to train and evaluate our CNN. We've encapsulated all of the code into a single function to make it really simple.

In [ ]:
run_experiment()

## LSTM Time Series Forecasting

The second task we will explore is using a long short-term memory (LSTM) model to perform time-series forecasting with our dataset. In other words, given the beginning portion of a time series, can we predict what the next $n$ time steps will be? This kind of application could be very useful to predict a seizure _before_ it happens. Since we now have a CNN model that can predict with high accuracy whether a time series is a seizure event, all we have to do is feed the time series forecast from the LSTM into the CNN classifier. If the LSTM can accurately predict what the next time points will be, and the CNN can accurately predict whether those time points represent a seizure event, then we will be well on our way to predicting seizures before they happen!

For now we will focus on _single-step_ forecasting, where we only predict one time point at a time. In practice we will need to predict many time steps at once in order to be fast enough for real-time applications.

Sources used for reference:
- https://towardsdatascience.com/using-lstms-to-forecast-time-series-4ab688386b1f
- https://github.com/kmsravindra/ML-AI-experiments/blob/master/AI/LSTM-time_series/LSTM%20-%20Sine%20wave%20predictor.ipynb

__NOTE__: The multi-step forecasting code (`n_steps > 1`) does not work yet. So that would be a good place to start if you're looking to work with this project!

In [ ]:
def lstm_create(window_size):
    # create lstm model
    model = keras.models.Sequential()
    model.add(keras.layers.LSTM(window_size, return_sequences=True, input_shape=(window_size, 1)))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.LSTM(256))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(1))
    model.add(keras.layers.Activation("linear"))

    # compile model
    model.compile(loss="mse", optimizer="adam")

    return model

def lstm_train(model, X_train, y_train):
    t1 = time.time()
    model.fit(X_train, y_train, batch_size=512, epochs=3, validation_split=0.1, verbose=False)
    t2 = time.time()

    print('train time: %f s' % (t2 - t1))

def lstm_predict(model, X_test, y_test, scaler, n_steps):
    # compute single-step prediction
    if n_steps == 1:
        y_pred = model.predict(X_test)

    # or compute multi-step prediction (note: does not work yet)
    else:
        y_pred = np.empty(n_steps)
        X_window = X_test[0:1, :]

        for i in range(n_steps):
            # compute single-step prediction
            y_pred_i = model.predict(X_window)
            y_pred_i = y_pred_i.reshape(1, 1, 1)

            # append single-step prediction to multi-step prediction
            y_pred[i] = y_pred_i[0, 0, 0]

            # update moving window
            X_window = np.concatenate([X_window[:, 1:, :], y_pred_i], axis=1)

    # get de-normalized ground truth, model predictions
    y_test = scaler.inverse_transform(y_test)
    y_pred = scaler.inverse_transform(y_pred)

    # report mean-squared error
    print('mse: %g' % (sklearn.metrics.mean_squared_error(y_test, y_pred)))

    return y_pred, y_test

def lstm_forecast(x, window_size=100, train_size=0.8, n_steps=1):
    # normalize input data to the range [-1, 1]
    scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(-1, 1))
    x = scaler.fit_transform(x)
    x = pd.DataFrame(x)

    # augment input data with time-shifted copies
    x_orig = x.copy()

    for i in range(window_size):
        x = pd.concat([x, x_orig.shift(-(i+1))], axis=1)

    # remove rows with missing data
    x.dropna(axis=0, inplace=True)

    # convert input data to numpy array
    x = x.values
    x = x.reshape(x.shape[0], x.shape[1], 1)

    # create train/test sets
    split_index = int(x.shape[0] * train_size)

    train = x[:split_index, :]
    test = x[split_index:, :]

    train = sklearn.utils.shuffle(train)

    X_train = train[:, :-1]
    y_train = train[:, -1]
    X_test = test[:, :-1]
    y_test = test[:, -1]

    # create lstm model
    model = lstm_create(window_size)

    # print model summary
    # model.summary()

    # train lstm model
    lstm_train(model, X_train, y_train)

    # compute forecast
    y_pred, y_test = lstm_predict(model, X_test, y_test, scaler, n_steps)

    return y_pred, y_test

def lstm_plot(y_test, y_pred, channel_name):
    s_rate = 256
    t = np.arange(len(y_test)) / s_rate

    plt.figure(figsize=(20, 6))
    plt.plot(t, y_test, label='Actual')
    plt.plot(t, y_pred, label='Predicted')
    plt.legend()
    plt.xlabel('Time (t) [s]')
    plt.ylabel('Voltage (V) [V]')
    plt.title('EEG Forecast for %s' % (channel_name))
    plt.show()

In [ ]:
# get channel csv filenames
csv_files = glob.glob('seizure.*.csv')

# initialize output arrays
channels = []
y_pred = []
y_test = []

# perform time series forecast for each channel
for filename in csv_files:
    # get channel name
    channel_name = filename.split('.')[-2]

    print(channel_name)

    # read channel dataframe
    df_channel = pd.read_csv(filename)

    # perform forecasting for a random sample
    column = random.choice(df_channel.columns)

    # extract channel k, sample i
    x_ki = pd.DataFrame(df_channel[column])

    # perform time series forecasting
    y_pred_ki, y_test_ki = lstm_forecast(x_ki)

    # append results
    channels.append(channel_name)
    y_pred.append(y_pred_ki)
    y_test.append(y_test_ki)

In [ ]:
# plot results
for k in range(len(y_pred)):
    lstm_plot(y_test[k], y_pred[k], channels[k])